<a href="https://colab.research.google.com/github/4may/Architectures/blob/master/12_factor_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 12 factor app

[slide shareの資料](https://www.slideshare.net/masatoshitada7/twelvefactor-app)に分かりやすそうなのがあったので参考にする。

[Javaでやってみる The Twelve Factor App JJUG-CCC 2014 Fall 講演資料](https://www.slideshare.net/nabedge/java-the-twelve-factor-app)も分かりやすそう。

## 12 factor appとは

クラウドアプリケーションが従うべき12個のベストプラクティスをまとめたもの。

* https://12factor.net/
* herokuが提唱
* プログラミング言語問わず
* [Patterns of Enterprise Application Architecture](https://books.google.co.jp/books/about/Patterns_of_enterprise_application_archi.html?id=FyWZt5DdvFkC&redir_esc=y)にインスパイア
  * 2003年！！
* 12個のアイディアは、大きく３つのグループに分けられる
  * portability
  * scaling
  * operation

## 原則

* 環境構築手順は**自動化**する。また、専用のフォーマットを使う。
* OSへの依存関係を明確にする。
* 計算機リソース(サーバー)管理は不要とする
* 開発環境と本番環境の差異は最小限にする
* **ビルドは一度だけ**にする。開発/テスト/本番環境のそれぞれで同じバイナリを使う

## Portability

### なぜ重要か

バイナリが、多様な環境下で動作するため。

例)

* 開発環境：ローカルPC
* テスト環境：オンプレミスのjenkins
* 本番環境:クラウド

### Portabilityに関するFactor

* Ⅲ. 設定 「設定は、環境変数に保存する」
* Ⅳ. バックエンドサービス 「バックエンドサービスはアタッチされたリソースとして扱う」
* Ⅶ. ポートバインディング 「ポートバインディングを通してサービスを公開する」

#### Ⅲ. 設定

* OSが違っても、環境変数の値を取得する方法は一緒
* 外部リソース(設定ファイルやDB)に保存しない。
  * バイナリに設定ファイルを含めてしまうと、環境ごとにビルドが必要
  * バイナリ外に設定ファイルを置くと、環境ごとにパスが異なる
  * DBに設定値を保存すると、DB接続が必要
  * 情報漏洩のリスク

#### Ⅳ. バックエンドサービス

* 「アタッチされたリソース」、とは「簡単に交換可能なリソース」を指す
* 簡単に交換可能なリソースとは？
  * ローカルPCのサービスへの接続とクラウドサービスへの接続を同じコードで行う
    * バイナリのリビルドしたくない
  * バックエンドサービスは代わりが効くようにする。

#### Ⅶ. ポートバインディング

* アプリケーション内に組込サーバーを持つべし
  * サーバーなしで実行できる

## Scalability

### Scale時の注意点

仮想マシン/コンテナは増減を繰り返す。**アプリが動く環境は短命。**

### Scalabilityに関するFactor

* Ⅷ. 並行性　「プロセスモデルによってスケールアウトする」
* Ⅵ. プロセス 「アプリケーションを１つ、ないしは複数のstatelessなプロセスとして実行する」
* Ⅺ. ログ　「ログをイベントストリームとして扱う」
* Ⅸ. 廃棄容易性　「高速な起動とgraceful shutdown」

#### Ⅷ.並行性

* 垂直スケールではなく、水平スケールで対応しよう
  * 垂直スケール：メモリ増やす、CPU強くする -> 環境の再起動が必要 -> 繋がらなくなる時間が発生
  * 水平スケール：インスタンスの数を増やす。 -> 繋がらなくなる時間は発生しない

#### Ⅵ. プロセス

* stateless : セッション、ローカルファイルは使わないようにする
  * 複数インスタンス
  * 仮想マシンがなくなる可能性

#### Ⅺ. ログ

* ログは標準出力に出す！
  * 仮想マシンはなくなる可能性がある -> ローカルファイルに出力すると、ログが消える可能性あり
  * windows/mac/linus, IDE/CLI, local/cloud問わず、動く
* ログは一箇所にまとめて出力する
  * [splunk](https://www.splunk.com/ja_jp)
  * [ELK stack](https://www.elastic.co/jp/what-is/elk-stack) : elasticsearch, logstash, kibanaの３つのサービスを組み合わせたログ解析環境

#### 廃棄容易性

* インスタンスの起動と停止はとにかく高速化する。スケールしやすくなる。
* graceful : 停止前の後始末を行うこと
  * 受信中リクエストの処理
  * リソース(DB接続など)の破棄

## Operation

### Operationミスの原因は？

* 手作業
* 環境による違い

**自動化 + 全環境で同一手順**を目指そう

### Operationミス防止に関するFactor

* X. 開発/本番一致　「開発/ステージング/本番環境をできるだけ一致させた状態を保つ」
* V. ビルド、リリース、実行　「ビルド、リリース、実行の３つのステージを厳密に分ける」
* Ⅻ. 管理プロセス　「管理タスクを1回限りのプロセスとして実行する」
* Ⅱ. 依存関係　「依存関係を明示的に宣言して、分離する」
* Ⅰ. コードベース　「バージョン管理されている１つのコードベースと複数のデプロイ」

#### Ⅹ. 開発/本番一致

* ギャップをなくす
 * 時間のギャップ：開発からデプロイまでの時間は**数時間以内**にする
 * 人材のギャップ：**開発者が運用もする**
 * フレームワークのギャップ：同じフレームワークを使う
  * 本番環境はOracle, 開発環境はPostgreSQLなどはNG

#### Ⅴ. ビルド、リリーズ、実行

* 本番環境で動いているコードを直にいじらない
  * テストできないので事故る
* 環境ごとにビルドし直さない
  * ポータブルでない
* 実行のたびにリリースを作り直さない

#### Ⅻ. 管理プロセス

* 管理プロセス(DBマイグレーションなど)は、**アプリの初期化処理**として行う
  * 起動時に自動で行われるようにする
* コマンド手打ちは極力しない

## Ⅱ. 依存関係

* アプリだけで全てを完結させる. アプリ外にあるライブラリやツールをあてにしない
  * アプリ外にあるリソースは常にあるとは限らない
  * ライブラリは、アプリの依存設定ファイルに列挙する
  * ツールは、アプリの中に含める

#### Ⅰ. コードベース

* 1アプリに1リポジトリとする
* １つのアプリを複数のリポジトリで管理しない
  * ビルドプロセスの明確化
* 1つのリポジトリに複数のアプリを置かない
  * 複数のアプリで共有する処理の密結合
* 複数環境は、一つのリポジトリで管理する